# SageMaker PyTorch 컨테이너를 이용한 하이퍼 파라미터 튜닝

Kernel `Python 3 (PyTorch CPU (or GPU) Optimized)` works well with this notebook.

## Contents

1. [배경](#배경)
1. [셋업](#셋업)
1. [데이터](#데이터)
1. [학습](#학습)
1. [호스트](#호스트)

---


## 배경

MNIST는 손으로 쓴 숫자 분류를 위해 널리 사용되는 데이터셋입니다. 이것은 손으로 쓴 숫자의 28x28 픽셀 그레이스케일 70,000개의 라벨로 구성되어 있습니다. 데이터 세트는 학습용 이미지 60,000개와 테스트 이미지 10,000개로 분할됩니다. 클래스는 10개(10자리당 하나씩)입니다. 이 튜토리얼은 PyTorch를 사용하여 SageMaker에서 MNIST 모델을 학습하고 테스트하는 방법을 보여줍니다. 또한 SageMaker 자동 모델 튜닝을 사용하여 최상의 모델을 얻기 위해 적절한 하이퍼 매개 변수를 선택하는 방법도 보여 줍니다.

SageMaker의 PyTorch에 대한 자세한 내용은 [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers)와 [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) 저장소를 방문하시기 바랍니다.

---

## 셋업

_이 노트북은 ml.m4.xlarge 노트북 인스턴스에서 생성 및 테스트되었습니다._

먼저 SageMaker 세션을 만들고 다음을 지정합니다.

- 학습 및 모델 데이터에 사용할 S3 버킷 및 접두사입니다. 노트북 인스턴스, 학습 및 호스팅과 동일한 영역 내에 있어야 합니다.
- IAM 역할 arn은 학습 및 호스팅을 통해 데이터에 대한 액세스를 제공합니다. 이러한 파일을 만드는 방법은 설명서를 참조합니다. 노트북 인스턴스, 학습 및/또는 호스팅에 둘 이상의 역할이 필요한 경우 `sagemaker.get_execution_role()`을 적절한 전체 IAM 역할 arn 문자열로 교체해 주십시오.


In [ ]:
import sagemaker
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist'

role = sagemaker.get_execution_role()


## 데이터
### 데이터 로드


In [ ]:
from torchvision import datasets, transforms

datasets.MNIST('data', download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
]))

### 데이터를 S3에 업로드

우리는 `sagemaker.Session.upload_data` 기능을 통해 S3 위치에 데이터셋을 업로드할 수 있습니다. 반환 값 입력은 위치를 식별합니다. 나중에 학습 작업을 시작할 때 사용합니다.


In [ ]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

## 학습
### 학습 스크립트

`mnist.py` 스크립트는 SageMaker 모델을 학습하고 호스팅하는 데 필요한 모든 코드(`model_fn` 기능)를 제공합니다.
학습 스크립트는 SageMaker 외부에서 실행할 수 있는 학습 스크립트와 매우 유사하지만 다음과 같은 다양한 환경 변수를 통해 학습 환경에 대한 유용한 속성에 액세스할 수 있습니다.

* `SM_MODEL_DIR`: 모델 아티팩트를 쓸 디렉터리의 경로를 나타내는 문자열입니다.
이러한 아티팩트는 모델 호스팅을 위해 S3에 업로드됩니다.
* `SM_NUM_GPUS`: 현재 컨테이너에서 사용할 수 있는 gpus 개수입니다.
* `SM_CURRENT_HOST`: 컨테이너 네트워크에 있는 현재 컨테이너의 이름입니다.
* `SM_HOSTS`: 모든 호스트를 포함하는 JSON 인코딩 목록입니다.

fit() 메서드에 대한 호출에서 하나의 입력 채널인 'training'(트레이닝)이 사용되었다고 가정하면, SM_CHANNEL_[channel_name] 형식에 따라 다음과 같이 설정됩니다.

* `SM_CHANNEL_TRAINING`: 'Training' 채널의 데이터를 포함하는 디렉토리의 경로를 나타내는 문자열입니다.

학습환경변수에 대한 자세한 내용은 [SageMaker Containers](https://github.com/aws/sagemaker-containers))를 방문하시기 바랍니다.

일반적인 학습 스크립트는 입력 채널에서 데이터를 로드하고, 하이퍼 파라미터로 학습을 구성하고, 모델을 학습하며, 모델을 `model_dir`에 저장하여 나중에 호스팅할 수 있도록 합니다. 하이퍼 파라미터는 인수로 스크립트에 전달되며 `argparse.ArgumentParser` 인스턴스를 사용하여 검색할 수 있습니다.

SageMaker는 학습 스크립트를 가져오기 때문에 이 예에서와 같이 동일한 스크립트를 사용하여 모델을 호스트하는 경우 학습 코드를 주 가드에 넣어야 합니다(``if __name__=='__main__':``). 따라서 SageMaker가 잘못된 실행 지점에서 학습 코드를 실수로 실행하지 않도록 해야 합니다.

예를 들어 이 노트북에서 실행되는 스크립트는 다음과 같습니다.

In [ ]:
!pygmentize mnist.py

### 하이퍼 파라미터 튜닝 작업 설정
*참고, 아래의 기본 설정을 사용하면 하이퍼 파라미터 조정 작업을 완료하는 데 약 20분이 걸릴 수 있습니다.*

이제 데이터 집합과 스크립트를 준비했으므로 모델을 학습할 준비가 되었습니다. 그러기 전에 주목할 점은 훈련된 모델의 성능에 극적으로 영향을 미칠 수 있는 많은 하이퍼 파라미터가 있다는 것입니다. 예를 들어, 학습 속도, 배치 크기, 에폭 수 등이 있습니다. 최상의 결과를 얻을 수 있는 하이퍼 매개 변수 설정은 데이터 집합에 따라 달라지기 때문에 최상의 하이퍼 매개 변수 설정을 검색하지 않고는 선택하기가 거의 불가능합니다. SageMaker 자동 모델 튜닝을 사용하여 하이퍼 파라미터 튜닝 작업을 생성하여 자동화되고 효과적인 방법으로 최상의 하이퍼 파라미터 설정을 검색할 수 있습니다.

이 예에서는 SageMaker Python SDK를 사용하여 하이퍼 파라미터 튜닝 작업을 설정하고 관리합니다. 특히, 조정하려는 각 하이퍼 파라미터에 대해 범주형 하이퍼 파라미터의 경우 가능한 값의 리스트 또는 범위를 지정합니다. 하이퍼 매개 변수 조정 작업은 서로 다른 하이퍼 매개 변수 설정으로 여러 학습 작업을 자동으로 시작하고 미리 정의된 "객관적 메트릭"을 기반으로 해당 학습 작업의 결과를 평가하며 이전 결과를 기반으로 향후 시도를 위한 하이퍼 매개 변수 설정을 선택합니다. 각 하이퍼 파라미터 조정 작업에 대해 예산(최대 학습 작업 수)을 부여하고 많은 학습 작업이 실행되면 완료됩니다.

이제 SageMaker Python SDK를 사용하여 하이퍼 파라미터 튜닝 작업을 설정합니다. 다음 단계를 따르십시오.
* 추정기를 생성하여 PyTorch 학습 작업을 설정합니다.
* 조정할 계획인 하이퍼 파라미터의 범위를 정의합니다. 이 예에서는 tuning learning_rate 및 batch size를 제공합니다.
* 최적화할 튜닝 작업의 목표 메트릭을 정의합니다.
* 위의 설정으로 하이퍼 파라미터 튜너를 만들고 리소스 구성을 조정합니다.

SageMaker에서 단일 PyTorch 작업을 학습하는 것과 마찬가지로 PyTorch 스크립트, IAM 역할 및 (작업당) 하드웨어 구성으로 전달되는 PyTorch Estimator를 정의합니다.

In [ ]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="mnist.py",
                    role=role,
                    framework_version='1.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.m4.xlarge',
                    hyperparameters={
                        'epochs': 6,
                        'backend': 'gloo'
                    })

estimator를 정의한 후에는 조정할 하이퍼 파라미터와 가능한 값을 지정할 수 있습니다. 우리는 세 가지 종류의 하이퍼 파라미터가 있습니다.
- 범주형 매개 변수는 이산형 집합에서 하나의 값을 가져와야 합니다. 가능한 값 목록을 `CategoricalParameter(list)`에 전달하여 이를 정의합니다.
- 연속 파라미터는 `ContinuousParameter(min, max)`로 정의된 최소값과 최대값 사이의 실제 숫자 값을 취할 수 있습니다.
- 정수 매개변수는 `IntegerParameter(min, max)`로 정의된 최소값과 최대값 사이의 정수 값을 취할 수 있습니다.

*가능한 경우 값을 최소 제한 유형으로 지정하는 것이 거의 항상 가장 좋습니다. 예를 들어 learning rate를 0.01에서 0.2 사이의 연속 값으로 튜닝하면 값이 0.01, 0.1, 0.15 또는 0.2인 범주형 매개 변수로 튜닝하는 것보다 더 좋은 결과를 얻을 수 있습니다. 일반적으로 2의 검정력이 권장되므로 배치 크기를 범주형 매개변수로 지정했습니다.*

In [ ]:
hyperparameter_ranges = {'lr': ContinuousParameter(0.001, 0.1),'batch-size': CategoricalParameter([32,64,128,256,512])}

다음에는 튜닝하고자 하는 목표 메트릭과 그 정의를 지정합니다. 여기에는 교육 작업의 CloudWatch 로그에서 메트릭을 추출하는 데 필요한 정규식(Regex)이 포함됩니다. 이 경우 스크립트는 평균 손실 값을 발생시키고 이를 목표 메트릭으로 사용할 것이며, 또한 목표_type을 '최소화'로 설정하여 하이퍼 파라미터 튜닝이 최상의 하이퍼 파라미터 설정을 검색할 때 목표 메트릭을 최소화하려고 합니다. 기본적으로 taral_type은 'maximize'로 설정됩니다.

In [ ]:
objective_metric_name = 'average test loss'
objective_type = 'Minimize'
metric_definitions = [{'Name': 'average test loss',
                       'Regex': 'Test set: Average loss: ([0-9\\.]+)'}]


이제 `HyperparameterTuner` 오브젝트를 만들겠습니다. 다음을 매개변수로 전달합니다.
- 위에서 만든 PyTorch Estimator
- 하이퍼 파라미터 범위
- 목표 메트릭 이름 및 정의
- 총적으로 실행할 교육 작업 수 및 병렬로 실행할 수 있는 교육 작업 수와 같은 리소스 구성을 조정

In [ ]:
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=9,
                            max_parallel_jobs=3,
                            objective_type=objective_type)

### 하이퍼 매개 변수 튜닝 작업을 시작합니다.
마지막으로 '.fit()'로 전화를 걸어 열차 및 테스트 데이터 세트로 가는 S3 경로를 통과함으로써 하이퍼프레미터 튜닝 작업을 시작할 수 있습니다.

하이퍼프리미터 튜닝 작업이 생성된 후에는 다음 단계에서 튜닝 작업을 설명하여 진행 상황을 확인할 수 있어야 하며, SageMaker 콘솔->Jobs로 이동하여 하이퍼 파라미터 튜닝 작업의 진행 상황을 확인할 수 있습니다.

In [ ]:
tuner.fit({'training': inputs})

## 호스트
### 엔드포인트 생성
학습 후에는 튜너 개체를 사용하여 `PyTorchPredictor`를 구축 및 배포합니다. 이렇게 하면 Sagemaker Endpoint가 만들어집니다. 이 서비스는 튜너의 최상의 모델을 기반으로 추론을 수행하는 데 사용할 수 있습니다. 이전 단계에서는 튜너가 튜닝 중에 여러 교육 작업을 시작했으며 최상의 목표 메트릭을 가진 결과 모델이 최상의 모델로 정의되었습니다.

위에서 언급한 바와 같이 `mnist.py`에 `model_fn` 구현하고 있습니다.필수 입력 사항입니다. [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers)에 정의된 `input_fn`, `predict_fn`, `output_fn`, `transform_fm`의 디폴트 구현을 사용합니다.

배포 함수에 대한 인수를 통해 엔드포인트에 사용할 인스턴스 수와 유형을 설정할 수 있습니다. 이 값은 학습 작업에 사용한 값과 같을 필요가 없습니다. 예를 들어 GPU 기반 인스턴스 집합에서 모델을 교육한 다음 CPU 기반 인스턴스 집합에 Endpoint를 배포할 수 있지만 모델을 `mnist.py`에서와 유사한 CPU 모델로 반환하거나 저장해야 합니다. 여기서는 이 모델을 단일 ```ml.m4.xlarge``` 인스턴스에 배치하겠습니다.


In [ ]:
predictor = tuner.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

### 평가
이제 이 예측 변수를 사용하여 손으로 쓴 숫자를 분류할 수 있습니다.

아래 셀을 실행하면 빈 이미지 상자가 표시됩니다. 그러면 그 안에 숫자를 그리면 픽셀 데이터가 이 노트북의 `data` 변수에 로드되고, 이 변수를 `predictor`로 전달할 수 있습니다.

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

In [ ]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

### 클린업

이 예제를 완료한 후에는 예측 엔드포인트를 삭제하여 관련 인스턴스를 해제해야 합니다.

In [ ]:
tuner.delete_endpoint()